In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import subprocess
import time
import os

# 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)

# 기본 URL 설정
url = 'https://gsc.gangseo.seoul.kr/meeting/confer/search.do#none'
driver.get(url)

# 검색어 입력
def input_search(keyword):
    search_input = driver.find_element(By.XPATH, '//*[@id="keyword"]')
    search_input.send_keys(keyword)

# 검색 버튼 클릭
def click_search_button():
    search_btn = driver.find_element(By.XPATH, '//*[@id="btnSearch"]')
    search_btn.click()

# 페이지 내 데이터 스크래핑 함수
def scrape_page(i, page_num_offset=0):
    try:
        # 페이지 네비게이션 버튼 클릭
        page_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="pagingNav"]/a[{i}]'))
        )
        driver.execute_script("arguments[0].click();", page_btn)
        print(f"페이지 {i-2 + page_num_offset} 이동 중...")

        # 페이지 로드 대기
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="sub_detail"]/div[2]/table/tbody/tr'))
        )
        print(f"✅ 페이지 {i-2 + page_num_offset} 로드 완료!")

        all_texts = []

        # tr 요소 찾기
        tr_elements = driver.find_elements(By.XPATH, '//*[@id="sub_detail"]/div[2]/table/tbody/tr')
        print(f"🔎 페이지 {i-2 + page_num_offset}에서 {len(tr_elements)}개의 tr 발견")

        for tr_index, tr in enumerate(tr_elements, start=1):
            try:
                tr_btn = tr.find_element(By.XPATH, './td[2]/a')  # <a> 태그 찾기
                print(f"✅ {tr_index}번째 tr 링크 발견! 이동 중...")

                # 링크 클릭
                if not tr_btn.is_displayed() or not tr_btn.is_enabled():
                    print(f"⚠️ tr {tr_index} 클릭 불가 (페이지 {i-2 + page_num_offset}) - 스킵")
                    continue

                # 새 창으로 이동
                before_windows = len(driver.window_handles)
                href = tr_btn.get_attribute("href")
                onclick = tr_btn.get_attribute("onclick")

                if href:
                    driver.execute_script("arguments[0].click();", tr_btn)
                elif onclick:
                    driver.execute_script(onclick)

                WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > before_windows)
                driver.switch_to.window(driver.window_handles[-1])

                # 내용 가져오기
                time.sleep(10)  # 페이지 로드 대기 시간 추가
                spk_elements = driver.find_elements(By.XPATH, '//*[@id="main-content"]//span')
                combined_text = " ".join([span.text for span in spk_elements if span.text.strip() != ""])
                all_texts.append(combined_text)

                # 창 닫기
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(1)

            except Exception as e:
                print(f"❌ 오류 발생 (페이지 {i-2 + page_num_offset}, tr {tr_index}): {e}")
        
        # 페이지별로 파일 저장
        save_page_data(all_texts, i-2 + page_num_offset)

        # 🔹 **이 페이지에서 `tr` 버튼이 10개 미만이면, 여기서 스크래핑을 종료!**
        if len(tr_elements) < 10:
            print(f"❌ 해당 페이지 {i-2 + page_num_offset}에서 tr 버튼이 10개 미만이므로 스크래핑을 종료합니다.")
            return False  # 반복문 종료 신호 반환
        return True  # 다음 페이지로 진행

    except Exception as e:
        print(f"❌ 페이지 {i-2 + page_num_offset} 오류 발생: {e}")
        return True  # 오류 발생 시 다음 페이지 진행 (스크래핑이 끊기지 않도록)

def save_page_data(all_texts, page_num):
    file_path = f"git@github.com:ryush930/web-scrapping.git/page{page_num}.txt"  # GitHub 저장소 경로로 변경
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write("\n\n".join(all_texts))  
    print(f"✅ 페이지 {page_num}의 데이터가 {file_path}에 저장되었습니다.")

    # GitHub에 파일 추가 및 커밋, 푸시
    commit_and_push(file_path, page_num)

def commit_and_push(file_path, page_num):
    try:
        # Git 명령어 실행
        subprocess.run(['git', 'add', file_path], check=True)
        subprocess.run(['git', 'commit', '-m', f'페이지 {page_num} 데이터 추가'], check=True)
        subprocess.run(['git', 'push'], check=True)
        print(f"✅ 페이지 {page_num}의 데이터가 GitHub에 업로드되었습니다.")
    except subprocess.CalledProcessError as e:
        print(f"❌ GitHub 업로드 중 오류 발생: {e}")

# 실행 부분
input_search("행정사무감사")  # 검색어 입력
click_search_button()  # 검색 버튼 클릭

# 페이지 1~10까지 네비게이션 실행
for i in range(3, 13):  # 페이지 3번부터 12번까지
    should_continue = scrape_page(i)
    if not should_continue:  # `tr` 버튼이 10개 미만이면 스크래핑 종료
        driver.quit()
        exit()

# 페이지 13 클릭하여 이후 페이지를 계속 스크래핑 (페이지 번호가 더 있을 때까지 반복)
try:
    page_num_offset = 10  # 페이지 번호 오프셋 (11~20부터 저장)
    while True:
        # 페이지 13 버튼 클릭
        page_13_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="pagingNav"]/a[13]'))
        )
        driver.execute_script("arguments[0].click();", page_13_btn)
        print(f"✅ 페이지 {page_num_offset+1} 이동 중...")

        # 페이지 3부터 시작 (실제로는 page 11부터 시작)
        for i in range(3, 13):
            should_continue = scrape_page(i, page_num_offset)
            if not should_continue:  # `tr` 버튼이 10개 미만이면 스크래핑 종료
                driver.quit()
                exit()

        page_num_offset += 10  # 다음 페이지 그룹으로 이동

except Exception as e:
    print(f"❌ 페이지 13 클릭 또는 이후 페이지 스크래핑 중 오류 발생: {e}")

# 스크래핑 완료 후 드라이버 종료
driver.quit()
